## SPARQL Query Examples

In [3]:
#Install the rdflib library
!pip install rdflib

In [1]:
#import necessary libraries
import rdflib   
import pandas as pd

In [2]:
# Load the RDF file
rdf_file = "cartographical_heritage_ontology.rdf" 
graph = rdflib.Graph()
graph.parse(rdf_file)

<Graph identifier=N014a9eb44e264e38a293819869ba12db (<class 'rdflib.graph.Graph'>)>

In [3]:
# Define the SPARQL query

#### Query 1
This query represents the fundamental relationship between the core entities in the ontology model, which are the different stages of the maps (F1, F2, F3, F5, D9) in their creation (F28), publication (F30), and digitization (D2).

In [6]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>
PREFIX cidoc-crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.cidoc-crm.org/extensions/crmdig/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?Title ?Shelfmark ?CONTENTdm_URL
WHERE {
  ?map_publication lrmoo:R24_created ?manifestation .
  ?map_creation lrmoo:R17_created ?expression .
  ?expression lrmoo:R3i_realises ?work .
  ?manifestation lrmoo:R4_embodies ?expression .
  ?item lrmoo:R7_exemplifies ?manifestation .
  ?item cidoc-crm:P48_has_preferred_identifier ?shelfmark_iri .
  ?shelfmark_iri rdfs:label ?Shelfmark .
  ?digitization crmdig:L1_digitized ?item .
  ?digitization crmdig:L20_has_created ?digital_image .
  ?digital_image cidoc-crm:P102_has_title ?title .
  ?title rdfs:label ?Title .
  ?digital_image cidoc-crm:P48_has_preferred_identifier ?CONTENTdm_URL .
}
"""

# Execute the query
results = graph.query(query)

# Process results into a DataFrame
data = []
for row in results:
    data.append({
        "Title": str(row.Title),
        "Shelfmark": str(row.Shelfmark),
        "CONTENTdm_URL": str(row.CONTENTdm_URL)
    })

df = pd.DataFrame(data)
df

,Title,Shelfmark,CONTENTdm_URL
0,Afbeeldinge der stadt Groningen met de omligge...,uklu 01-28-18,http://cdm21053.contentdm.oclc.org/cdm/ref/col...
1,Tweede stuk der beleegering van Bergen op Zoom...,uklu 01-23-25,http://cdm21053.contentdm.oclc.org/cdm/ref/col...
2,Grand theatre de la guerre sur les frontieres ...,uklu 01-22-03,http://cdm21053.contentdm.oclc.org/cdm/ref/col...
3,Italia,Rol middel 003,http://cdm21053.contentdm.oclc.org/cdm/ref/col...
4,Europae Tabula IIII,uklu RD--- 1,http://cdm21053.contentdm.oclc.org/cdm/ref/col...
5,Wereldkaart 1509,uklu RG 1,http://cdm21053.contentdm.oclc.org/cdm/ref/col...


#### Query 2
This query extracts basic provenance information (creator, publisher, spacetime information for publication and creation) from the RDF data.

In [8]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>
PREFIX cidoc-crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.cidoc-crm.org/extensions/crmdig/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?Shelfmark ?Publication_Date ?Publication_Place ?publisher_id ?Publisher ?Creation_Date ?creator_id ?Creation_Place ?Creator
WHERE {
  ?map_publication lrmoo:R24_created ?manifestation .
  ?map_creation lrmoo:R17_created ?expression .
  ?expression lrmoo:R3i_realises ?work .
  ?manifestation lrmoo:R4_embodies ?expression .
  ?item lrmoo:R7_exemplifies ?manifestation .
  ?item cidoc-crm:P48_has_preferred_identifier ?shelfmark_iri .
  ?shelfmark_iri rdfs:label ?Shelfmark .
  OPTIONAL {
    ?map_publication cidoc-crm:P4_has_time-span ?pub_timespan .
    ?pub_timespan cidoc-crm:P82_at_some_time_within ?Publication_Date .
  }
  OPTIONAL {
    ?map_creation cidoc-crm:P4_has_time-span ?cre_timespan .
    ?cre_timespan cidoc-crm:P82_at_some_time_within ?Creation_Date .
  } 
  OPTIONAL {
    ?map_publication cidoc-crm:P14_carried_out_by ?publisher_id .
    ?publisher_id rdfs:label ?Publisher .
  }
  OPTIONAL {
    ?map_creation cidoc-crm:P14_carried_out_by ?creator_id .
    ?creator_id rdfs:label ?Creator .
  } 
  OPTIONAL {
    ?map_publication cidoc-crm:P7_took_place_at ?publication_place .
    ?publication_place rdfs:label ?Publication_Place .
  }
  OPTIONAL {
    ?map_creation cidoc-crm:P7_took_place_at ?creation_place .
    ?creation_place rdfs:label ?Creation_Place .
  }
}
"""

results = graph.query(query)
data = []
for row in results:
    data.append({
        "Shelfmark": str(row.Shelfmark),
        "Publication_Date": str(row.Publication_Date) if row.Publication_Date else None,
        "Publication_Place": str(row.Publication_Place) if row.Publication_Place else None,
        "publisher_id": str(row.publisher_id) if row.publisher_id else None,
        "Publisher": str(row.Publisher) if row.Publisher else None,
        "Creation_Date": str(row.Creation_Date) if row.Creation_Date else None,
        "creator_id": str(row.creator_id) if row.creator_id else None,
        "Creation_Place": str(row.Creation_Place) if row.Creation_Place else None,
        "Creator": str(row.Creator) if row.Creator else None
    })

df = pd.DataFrame(data)
df

,Shelfmark,Publication_Date,Publication_Place,publisher_id,Publisher,Creation_Date,creator_id,Creation_Place,Creator
0,uklu 01-28-18,1694,None,None,None,1652,https://isni.org/isni/000000005731997X,None,Egbert Haubois
1,uklu 01-23-25,1747,Den Haag,https://isni.org/isni/0000000389329146,Daniel Langeweg,None,https://isni.org/isni/0000000389329146,None,Daniel Langeweg
2,uklu 01-22-03,1740-1749,Amsterdam,https://isni.org/isni/http://www.cidoc-crm.org...,"Chez Jean Covens et Corneille Mortier, Geographes",None,https://isni.org/isni/0000000121232028,None,Guillaume Delisle
3,Rol middel 003,1984,Gotha,https://isni.org/isni/0000000123311600,Hermann Haack,None,https://isni.org/isni/0000000072511492,None,Albert Kampen
4,uklu RD--- 1,1605,Amsterdam,https://isni.org/isni/http://www.cidoc-crm.org...,J. Hondius en C. Claesz,None,None,None,None
5,uklu RG 1,1509,Paris,https://isni.org/isni/0000000121358683,Henricum Stephanum,1503,https://isni.org/isni/0000000122762987,None,Gregor Reisch


#### Query 3
This query searches maps that depict places within the Netherlands.

In [9]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>
PREFIX cidoc-crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.cidoc-crm.org/extensions/crmdig/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?Identifier ?place_id ?place 
WHERE {
  ?parent_place cidoc-crm:P89i_contains ?place_id .
  ?parent_place rdfs:label "Netherlands" .
  ?place_id rdfs:label ?place .
  ?map cidoc-crm:P138_represents ?place_id .
  ?map cidoc-crm:P48_has_preferred_identifier ?Identifier .
}
"""

results = graph.query(query)
data = []
for row in results:
    data.append({
        "Identifier": str(row.Identifier),
        "place_id": str(row.place_id),
        "place": str(row.place)
    })

df = pd.DataFrame(data)
df

,Identifier,place_id,place
0,http://cdm21053.contentdm.oclc.org/cdm/ref/col...,https://www.geonames.org/2750250,Nieuweschans
1,http://cdm21053.contentdm.oclc.org/cdm/ref/col...,https://www.geonames.org/2755251,Groningen (city)
2,http://cdm21053.contentdm.oclc.org/cdm/ref/col...,https://www.geonames.org/2755812,Friesland
3,http://cdm21053.contentdm.oclc.org/cdm/ref/col...,https://www.geonames.org/2757339,Delfzijl
4,http://cdm21053.contentdm.oclc.org/cdm/ref/col...,https://www.geonames.org/2757936,Coevorden
5,http://cdm21053.contentdm.oclc.org/cdm/ref/col...,https://www.geonames.org/2758514,Bourtange
6,http://cdm21053.contentdm.oclc.org/cdm/ref/col...,https://www.geonames.org/2759144,Bergen op Zoom (city)
7,http://cdm21053.contentdm.oclc.org/cdm/ref/col...,https://www.geonames.org/2759875,Rijn


#### Query 4
This query extracts linguistic features, language, and IIIF image links.

In [10]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>
PREFIX cidoc-crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.cidoc-crm.org/extensions/crmdig/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?Identifier ?Title ?Description ?Language ?IIIF_Image 
WHERE {
  ?expression cidoc-crm:P165_incorporates ?linguisticItem .  
  ?linguisticItem rdf:type cidoc-crm:E33_Linguistic_Object .
  ?linguisticItem rdfs:label ?Description .
  OPTIONAL { 
    ?linguisticItem cidoc-crm:P72_has_language ?language .
    ?language rdf:type cidoc-crm:E56_Language .
    ?language rdfs:label ?Language .
  }
  ?manifestation lrmoo:R4_embodies ?expression .
  ?item lrmoo:R7_exemplifies ?manifestation .
  ?digitization crmdig:L1_digitized ?item .
  ?digitization crmdig:L20_has_created ?Digital_Image .
  ?Digital_Image cidoc-crm:P102_has_title ?title .
  ?title rdf:type cidoc-crm:E35_Title .
  ?title rdfs:label ?Title .
  ?Digital_Image cidoc-crm:P48_has_preferred_identifier ?Identifier .
  ?IIIF_Image cidoc-crm:P138_represents ?Digital_Image . 
}
"""

results = graph.query(query)
data = []
for row in results:
    data.append({
        "Identifier": str(row.Identifier),
        "Title": str(row.Title),
        "Description": str(row.Description),
        "Language": str(row.Language) if row.Language else None,
        "IIIF_Image": str(row.IIIF_Image)
    })

df = pd.DataFrame(data)
df

,Identifier,Title,Description,Language,IIIF_Image
0,http://cdm21053.contentdm.oclc.org/cdm/ref/col...,Tweede stuk der beleegering van Bergen op Zoom...,Legenda,None,https://dbc.rug.nl/digital/iiif/Kaarten/1489/f...
1,http://cdm21053.contentdm.oclc.org/cdm/ref/col...,Afbeeldinge der stadt Groningen met de omligge...,Met lijsten van openbare gebouwen en gasthuize...,None,https://dbc.rug.nl/digital/iiif/Kaarten/1223/f...
2,http://cdm21053.contentdm.oclc.org/cdm/ref/col...,Tweede stuk der beleegering van Bergen op Zoom...,Title in map,Italian,https://dbc.rug.nl/digital/iiif/Kaarten/1489/f...


#### Query 5
This query extracts visual features and IIIF image links of the maps.

In [11]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>
PREFIX cidoc-crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.cidoc-crm.org/extensions/crmdig/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?Identifier ?Title ?VisualFeature ?IIIF_Image
WHERE {
  ?expression cidoc-crm:P165_incorporates ?visualitem .  
  ?visualitem rdf:type cidoc-crm:E36_Visual_Item .
  ?visualitem rdfs:label ?VisualFeature .
  ?manifestation lrmoo:R4_embodies ?expression .
  ?item lrmoo:R7_exemplifies ?manifestation .
  ?digitization crmdig:L1_digitized ?item .
  ?digitization crmdig:L20_has_created ?Digital_Image .
  ?Digital_Image cidoc-crm:P102_has_title ?title .
  ?title rdf:type cidoc-crm:E35_Title .
  ?title rdfs:label ?Title .
  ?Digital_Image cidoc-crm:P48_has_preferred_identifier ?Identifier .
  ?IIIF_Image cidoc-crm:P138_represents ?Digital_Image .
}
"""

results = graph.query(query)
data = []
for row in results:
    data.append({
        "Identifier": str(row.Identifier),
        "Title": str(row.Title),
        "VisualFeature": str(row.VisualFeature),
        "IIIF_Image": str(row.IIIF_Image)
    })

df = pd.DataFrame(data)
df

,Identifier,Title,VisualFeature,IIIF_Image
0,http://cdm21053.contentdm.oclc.org/cdm/ref/col...,Afbeeldinge der stadt Groningen met de omligge...,portretten van stadhouders en de wapens van pr...,https://dbc.rug.nl/digital/iiif/Kaarten/1223/f...
1,http://cdm21053.contentdm.oclc.org/cdm/ref/col...,Wereldkaart 1509,vier windhoofden,https://dbc.rug.nl/digital/iiif/Kaarten/2791/f...


#### Query 6
This query extracts bibliographical references from the RDF data, distinguishing between antique and modern references.

In [13]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>
PREFIX cidoc-crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.cidoc-crm.org/extensions/crmdig/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?Shelfmark ?Reference_Title_antique ?Author_id_antqiue ?Author_name_antique ?Reference_Title_modern ?Author_id_modern ?Author_name_modern
WHERE {
  ?map_publication lrmoo:R24_created ?manifestation .
  ?map_creation lrmoo:R17_created ?expression .
  ?expression lrmoo:R3i_realises ?work .
  ?manifestation lrmoo:R4_embodies ?expression .
  ?item lrmoo:R7_exemplifies ?manifestation .
  ?item cidoc-crm:P48_has_preferred_identifier ?shelfmark_iri .
  ?shelfmark_iri rdfs:label ?Shelfmark .
  
  OPTIONAL {
    ?antique_reference cidoc-crm:P46_is_composed_of ?item .
    ?antique_reference cidoc-crm:P102_has_title ?ant_ref_title .
    ?ant_ref_title rdfs:label ?Reference_Title_antique .
    ?antique_reference lrmoo:R7_exemplifies ?ant_manifestation .
    ?ant_manifestation lrmoo:R4_embodies ?ant_expression .
    ?ant_creation lrmoo:R17_created ?ant_expression .
    ?ant_creation cidoc-crm:P14_carried_out_by ?Author_id_antqiue .
    ?Author_id_antqiue rdfs:label ?Author_name_antique .
  }
  
  OPTIONAL {
    ?reference_page cidoc-crm:P130_shows_features_of ?item .
    ?reference_page rdfs:label ?Reference .
    ?modern_reference cidoc-crm:P46_is_composed_of ?reference_page .
    ?modern_reference cidoc-crm:P102_has_title ?md_ref_title .
    ?md_ref_title rdfs:label ?Reference_Title_modern .
    ?modern_reference lrmoo:R7_exemplifies ?md_manifestation .
    ?md_manifestation lrmoo:R4_embodies ?md_expression .
    ?md_creation lrmoo:R17_created ?md_expression .
    ?md_creation cidoc-crm:P14_carried_out_by ?Author_id_modern .
    ?Author_id_modern rdfs:label ?Author_name_modern .
  }
}
"""

results = graph.query(query)

data = []
for row in results:
    data.append({
        "Shelfmark": str(row.Shelfmark),
        "Reference_Title_antique": str(row.Reference_Title_antique) if row.Reference_Title_antique else None,
        "Author_id_antqiue": str(row.Author_id_antqiue) if row.Author_id_antqiue else None,
        "Author_name_antique": str(row.Author_name_antique) if row.Author_name_antique else None,
        "Reference_Title_modern": str(row.Reference_Title_modern) if row.Reference_Title_modern else None,
        "Author_id_modern": str(row.Author_id_modern) if row.Author_id_modern else None,
        "Author_name_modern": str(row.Author_name_modern) if row.Author_name_modern else None
    })

df = pd.DataFrame(data)
df

,Shelfmark,Reference_Title_antique,Author_id_antqiue,Author_name_antique,Reference_Title_modern,Author_id_modern,Author_name_modern
0,uklu 01-28-18,None,None,None,None,None,None
1,uklu 01-23-25,None,None,None,"Atlas van Stolk : katalogus der historie-, spo...",https://isni.org/isni/0000000047415407,Gerrit van Rijn
2,uklu 01-22-03,None,None,None,Atlantes Neerlandici,https://isni.org/isni/0000000109366197,C. Koeman
3,Rol middel 003,None,None,None,None,None,None
4,uklu RD--- 1,Claudii Ptolemæi Alexandrini geographiae libri...,https://isni.org/isni/0000000051186344,Claudius Ptolemaeus,None,None,None
5,uklu RG 1,None,None,None,None,None,None
